In [2]:
import os


In [ ]:
#Reserve a RAM percentage for theano (cnmem = 80%)
os.environ['THEANO_FLAGS']='mode=FAST_RUN,device=gpu0,lib.cnmem=0.8,nvcc.fastmath=True,floatX=float32'

In [1]:
#np.random.seed(1337)  # for reproducibility  - (OJO: incorrect initialization can slow down or even completely stall the learning process.)

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, BatchNormalization
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD
import keras.callbacks as callbacks
from keras.regularizers import l2

import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from numpy import genfromtxt
import retinaMethods as rm
import kappa_methods as kappa
from generators_grayPre import (BatchGenerator ,random_rotation, ValidationGenerator, OriginalDataGenerator, BatchValidationGenerator)
import paths_file


Using Theano backend.
Using gpu device 0: GeForce GTX 980 Ti (CNMeM is disabled, cuDNN 4007)


In [3]:
#Data info
# mean_RGB = [100.009405273 ,100.227072634 , 100.786751717] #Not used 
# std_RGB = [84.3177751745 , 84.2576168412, 84.4614340298] #Not used 

#Network parameters
nb_classes = 5
# input image dimensions
img_rows = img_cols = 512

layers = 3
# convolution kernel size
kernel_size = (3, 3) #3×3 kernel, stride 1 and padding 1.
# size of pooling area for max pooling
pool_size = (3, 3) # the pooling window is 3×3, and the stride is 2
pool_stride = (2,2)


#Verify if the backed of keras is Theano ('th') or TensorFlow ("tf") or Theano ('th'). 2016-11-17_eph_131_acc_0.49
if K.image_dim_ordering() == 'th':
#     X_train = X_train.reshape(X_train.shape[0], layers, img_rows, img_cols) #THE DATA IS ALREADY IN THIS FORM
#     X_test = X_test.reshape(X_test.shape[0], layers, img_rows, img_cols)
    input_shape = (layers, img_rows, img_cols)
    print('Using theano')

else:
#     X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, layers)
#     X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, layers)
    input_shape = (img_rows, img_cols, layers)
    print('Using tensorflow')
    

Using theano


In [4]:
# we follow the convolutional layer by batch normalization layer and ReLu activations.. batch normalization????????
model = Sequential()

model.add(Convolution2D(16, kernel_size[0], kernel_size[1], border_mode='same', activation='relu', input_shape=input_shape)) #,  W_regularizer=l2(0.01)))
#Batch Normalization, 0: feature-wise normalization for the filtered images -(1: sample-wise normalization.)
model.add(BatchNormalization(mode = 0)) 
model.add(Convolution2D(16, kernel_size[0], kernel_size[1], border_mode='same', activation='relu' )) #,  W_regularizer=l2(0.01)))
model.add(BatchNormalization(mode = 0)) 


model.add(MaxPooling2D(pool_size=pool_size , strides=pool_stride))

model.add(Convolution2D(32, kernel_size[0], kernel_size[1], border_mode='same', activation='relu'  )) #,  W_regularizer=l2(0.01)))
model.add(BatchNormalization(mode = 0)) 
model.add(Convolution2D(32, kernel_size[0], kernel_size[1], border_mode='same', activation='relu' )) #, W_regularizer=l2(0.01) ))
model.add(BatchNormalization(mode = 0)) 

model.add(MaxPooling2D(pool_size=pool_size , strides=pool_stride))

model.add(Convolution2D(64, kernel_size[0], kernel_size[1], border_mode='same', activation='relu' )) #,  W_regularizer=l2(0.01)))
model.add(BatchNormalization(mode = 0)) 
model.add(Convolution2D(64, kernel_size[0], kernel_size[1], border_mode='same', activation='relu' )) #,  W_regularizer=l2(0.01)))
model.add(BatchNormalization(mode = 0)) 

model.add(MaxPooling2D(pool_size=pool_size , strides=pool_stride))

model.add(Convolution2D(96, kernel_size[0], kernel_size[1], border_mode='same', activation='relu' )) #,  W_regularizer=l2(0.01)))
model.add(BatchNormalization(mode = 0)) 

model.add(MaxPooling2D(pool_size=pool_size , strides=pool_stride))

model.add(Convolution2D(96, kernel_size[0], kernel_size[1], border_mode='same', activation='relu' )) #, W_regularizer=l2(0.01)))
model.add(BatchNormalization(mode = 0)) 

model.add(MaxPooling2D(pool_size=pool_size , strides=pool_stride))

model.add(Convolution2D(128, kernel_size[0], kernel_size[1], border_mode='same', activation='relu' ))
model.add(BatchNormalization(mode = 0)) 

model.add(MaxPooling2D(pool_size=pool_size , strides=pool_stride))

model.add(Dropout(0.25)) 
model.add(Flatten())
model.add(Dense(96, activation='relu' , W_regularizer=l2(0.01))) #Relu . Saturated in cero for numbers lower tha 0
model.add(Dropout(0.25))  
model.add(BatchNormalization(mode = 0)) 
model.add(Dense(nb_classes, activation='softmax' ))#,  W_regularizer=l2(0.01)))

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 16, 512, 512)  448         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
batchnormalization_1 (BatchNormal(None, 16, 512, 512)  1024        convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 16, 512, 512)  2320        batchnormalization_1[0][0]       
____________________________________________________________________________________________________
batchnormalization_2 (BatchNormal(None, 16, 512, 512)  1024        convolution2d_2[0][0]            
___________________________________________________________________________________________

In [ ]:
                                    #TAKER CARE!!! RUN MOUNT DATA FIRST IN CMD!!
from keras.models import model_from_json

json_string = model.to_json()
# model = model_from_json(json_string)

filepath= paths_file.json_models_path +"Json"+ str( datetime.now().strftime("%Y-%m-%d"))+".txt"

text_file = open(filepath, "w")  
text_file.write(json_string)
text_file.close()

In [ ]:
# let's train the model using SGD + momentum (how original).
#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#Loss: multiclass logloss as a loss function or Quadratic weighted kappa (QWK)
model.compile(loss= 'categorical_crossentropy',  #kappa.quad_kappa_log_hybrid_loss_clipped, # 
              optimizer='adadelta',
              metrics=['accuracy' , kappa.quad_kappa])
#lr = OJO: Higher learning rates will decay the loss faster, but they get stuck at worse values of loss 
print("Compiled")

In [5]:
# load Weights -------------------------------------------OJO !!!!

#load Weights 
# weights = paths_file.net_weights_path  + str("final_2016-11-17_kappa_test_0.70.hdf5")
# weights = paths_file.net_weights_path  + str("final_2016-11-25_kappa_test_0.752.hdf5")
#load Weights 
# model.load_weights(weights)

In [ ]:
class PlotAcc(callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []
        self.val_acc = []
        self.loss = []
        self.val_loss = []
        self.kappa = []
        self.val_kappa = []
#         print("logs: ",logs)


    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
        self.loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))
        self.kappa.append(logs.get('quad_kappa'))
        self.val_kappa.append(logs.get('val_quad_kappa'))
        


        if(len(self.acc) > 1):
             # summarize history for acc
            plt.plot(self.acc)
            plt.plot(self.val_acc)
            plt.title('model accuracy' ) #- %d epochs'% (e+1))
            plt.ylabel('accuracy')
            plt.xlabel('epochs')
            plt.legend(['train', 'test'], loc='upper left')
            plt.show()
            # summarize history for kappa
            plt.plot(self.kappa)
            plt.plot(self.val_kappa)
            plt.title('Quadratic kappa' )# - %d epochs'% (e+1))
            plt.ylabel('quad kappa')
            plt.xlabel('epochs')
            plt.legend(['train', 'test'], loc='upper left')
            plt.show()
            # summarize history for loss
            plt.plot(self.loss)
            plt.plot(self.val_loss)
            plt.title('model loss' )# - %d epochs'% (e+1))
            plt.ylabel('loss')
            plt.xlabel('epochs')
            plt.legend(['train', 'test'], loc='upper left')
            plt.show()

In [ ]:

#TRAINING PROCESS
print("Initial time: " , datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

batch_size = 16 #16  #to fit in memory  (32 not enough memory)
samples_per_epoch = 10000 #35100
nb_epoch = 200
#For validation 
batch_test = 16 
amount_val_samples = 3000 #3000

plotBehavior = PlotAcc()

# checkpoint
filepath= paths_file.net_weights_path  + str( datetime.now().strftime("%Y-%m-%d"))+"_eph_{epoch:03d}_acc_{val_acc:.2f}_val_quad_kappa_{val_quad_kappa:.2f}.hdf5"  #""weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='max')
#Callbacks list
callbacks_list = [checkpoint , plotBehavior]
# Fit the model
#Train uniform, val not uniform
# history = model.fit_generator(BatchGenerator(batch_size, nb_classes) , samples_per_epoch , nb_epoch , verbose=1, callbacks=callbacks_list, validation_data= ValidationGenerator(batch_test ,nb_classes), nb_val_samples= amount_val_samples)
#Train and val not uniform
# history = model.fit_generator(OriginalDataGenerator(batch_size, nb_classes) , samples_per_epoch , nb_epoch , verbose=1, callbacks=callbacks_list, validation_data= ValidationGenerator(batch_size, nb_classes), nb_val_samples= amount_val_samples)
#Train and val uniform
history = model.fit_generator(BatchGenerator(batch_size, nb_classes) , samples_per_epoch , nb_epoch , verbose=1, \
                              callbacks=callbacks_list, validation_data= BatchValidationGenerator(batch_test ,nb_classes), nb_val_samples= amount_val_samples)


#Save the history in variables
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
 

print("End time: " , datetime.now().strftime("%Y-%m-%d %H:%M:%S"))


In [6]:
#Kappa fot a set of 100 Test IMAGES

testPath = paths_file.test_processed_path
X_test = np.load(os.path.join(testPath, "images_200.npy"))  
y_test = np.load(os.path.join(testPath, "labels_200.npy")) 

for i in np.arange(X_test.shape[0]):
    #Rotate randomly all the images
    X_test[i] = random_rotation(X_test[i], 360, fill_mode = 'constant', cval=128)
#Get network output
proba = model.predict_proba(X_test)
#print(proba)
#Get correspongig lable to the output
classes = model.predict_classes(X_test , batch_size= 100)

k = kappa.kappa(y_test , classes , nb_classes)

print("Kappa: " + str(k))

99/99 [==============================] - 2s
Kappa: 0.771757918548


In [7]:
classes

array([0, 0, 0, 2, 2, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 0,
       2, 2, 0, 0, 3, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 3, 2, 2, 0, 0, 3, 3, 0,
       0, 2, 2, 0, 0, 2, 0, 2, 2, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0,
       0, 0, 0, 0, 0, 0, 0])

In [8]:
#Kappa for all the images

testPath = paths_file.test_processed_path

all_imag = [f for f in os.listdir(testPath) if f.startswith('i')]
all_label = [f for f in os.listdir(testPath) if f.startswith('l')]

#Sort the files names to be according with the labeling file
rm.sort_nicely(all_imag) 
rm.sort_nicely(all_label) 

pred_classes = np.array([]).astype('int')
y_test = np.array([]).astype('int')

print("Initial time: " , datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

for i in xrange(len(all_imag)):
          
    #Read set of data
    X_test = np.load(os.path.join(testPath, all_imag[i]))
    
    y_test = np.append(y_test , np.load(os.path.join(testPath, all_label[i])), axis = 0)
    
    #Normaliztion               
    #Aply random rotation to the test images to make them similar to the way in wich the net was trained 
#     for i in np.arange(X_test.shape[0]):
#         #Rotate randomly all the images
#         X_test[i] = random_rotation(X_test[i], 360, fill_mode = 'constant', cval=128)
    #Get correspongig lable to the output
    pred_classes = np.append(pred_classes, model.predict_classes(X_test  , batch_size= 100), axis = 0)

    del(X_test)
            
k = kappa.kappa(y_test , pred_classes , nb_classes)
print("End time: " , datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

print("Kappa test: " + str(k))


('Initial time: ', '2016-11-28 10:45:14')
76/76 [==============================] - 1s
('End time: ', '2016-11-28 11:13:58')
Kappa test: 0.752653249556


In [ ]:
#PLOT any image
array = X_test

print(array.shape)

array = np.rollaxis(array,1,4)
print(array.shape)
img = array[5,:,:,:]
plt.imshow(img)
plt.show()

In [ ]:
#Kappa fot a set of 100 Test IMAGES
import numpy as np
import os
import retinaMethods as rm

testPath =  paths_file.test_processed_path
X_test = np.load(os.path.join(testPath, "images_200.npy"))  
y_test = np.load(os.path.join(testPath, "labels_200.npy")) 

labelsNum = y_test
prob = []
print("Original data distribution:")
for k in range(5):
    print(np.sum(labelsNum==k)/float(labelsNum.size)*100)
    prob.append(np.sum(labelsNum==k)/float(labelsNum.size))
    
prob = np.array(prob)
